### PROBLEM 3 : Gaussian Mixture on toy data
##### You are required to implemet the main EM loop, but can use math API/functions provided by your language to calculate normal densities, covariance matrix, etc.
- The gaussian 2-dim data on file  2gaussian.txt  has been generated  using a mixture  of  two Gaussians, each  2-dim, with the parameters below. Run the EM algorithm with random initial values to recover the parameters.
    - mean_1 [3,3]); cov_1 = [[1,0],[0,3]]; n1=2000 points
    - mean_2 =[7,4]; cov_2 = [[1,0.5],[0.5,1]]; ; n2=4000 points
    
You should obtain a result visually like this (you dont necessarily have to plot it)

- Same problem for 2-dim data on file 3gaussian.txt , generated using a mixture of three Gaussians. Verify your  findings against the true parameters used generate the data below.
    - mean_1 = [3,3] ; cov_1 = [[1,0],[0,3]]; n1=2000
    - mean_2 = [7,4] ; cov_2 = [[1,0.5],[0.5,1]] ; n2=3000
    - mean_3 = [5,7] ; cov_3 = [[1,0.2],[0.2,1]]    ); n3=5000

----

Importing dataset 2gaussian in a numpy array

In [1]:
import numpy as np
import matplotlib.pyplot as plt

inputFile = open("./2gaussian.txt", 'r')
num_lines = len(inputFile.readlines())

dataset = np.zeros((num_lines, 2))
inputFile = open("./2gaussian.txt", 'r')
ind = 0
for line in inputFile:
    points = line.strip().split(" ")
    x = points[0]
    y = points[1]
    dataset[ind][0] = x
    dataset[ind][1] = y
    ind += 1

# print("x: ", dataset[:,0])
# print("y: ", dataset[:,1])
# plt.scatter(dataset[:,0], dataset[:,1])
# plt.show()

GMM implementation

In [34]:
'''
Gaussian Mixture Model implementation in python. This implementations is inspired from Oran looney's implementation
(ref. https://www.oranlooney.com/post/ml-from-scratch-part-5-gmm/)
'''

import numpy as np
from scipy.stats import multivariate_normal

class GaussianMixtureModel:
    def __init__(self, k=2, max_iter=50):
        self.k = k
        self.max_iter = max_iter

    def fit(self, X):
        self.num_samples, self.num_features = X.shape
        
        #initialization
        self.pi = np.full(shape=(self.num_samples, self.k), fill_value=1)
        self.weights = np.full(shape=(self.num_samples, self.k), fill_value=1)
        self.mean = X[np.random.choice(self.num_samples, self.k, replace=False)]
        self.covariance = [ [1,1] for i in range(self.k) ]

        for i in range(self.max_iter):
            self.e_step(X)
            self.m_step(X)

        return self.mean, self.covariance, self.pi

    def e_step(self, X):
        likelihood = np.zeros( (self.num_samples, self.k) )
        for m in range(self.k):
            likelihood[:,m] = multivariate_normal(mean=self.mean[m], cov=self.covariance[m]).pdf(X)
        self.weights = (likelihood * self.pi)/(likelihood * self.pi).sum(axis=1)[:, np.newaxis]
        self.pi = self.weights.mean(axis=0)
        

    def m_step(self, X):
        for i in range(self.k):
            weight = self.weights[:, [i]]
            total_weight = weight.sum()

            self.covariance[i] = np.cov(X.T, 
                aweights=(weight/total_weight).flatten(), 
                bias=True)
            self.mean[i] = (X * weight).sum(axis=0) / total_weight
            


Running GMM on 2gaussian dataset

In [35]:
gmm = GaussianMixtureModel(k=2)
mean, cov, pi = gmm.fit(dataset)

print("Mean 1: ", mean[0], " Covariance 1: ", cov[0], "N1: ", pi[0])
print("\nMean 2: ", mean[1], " Covariance 2: ", cov[1], "N2: ", pi[1])


Mean 1:  [2.99413206 3.05209657]  Covariance 1:  [[1.01023465 0.02719135]
 [0.02719135 2.93782278]] N1:  0.33479580743558174

Mean 2:  [7.01314844 3.98313426]  Covariance 2:  [[0.97475871 0.49747019]
 [0.49747019 1.00114249]] N2:  0.665204192564418


Importing dataset 3gaussian in a numpy array

In [36]:
import numpy as np
import matplotlib.pyplot as plt

inputFile = open("./3gaussian.txt", 'r')
num_lines = len(inputFile.readlines())

dataset2 = np.zeros((num_lines, 2))
inputFile = open("./3gaussian.txt", 'r')
ind = 0
for line in inputFile:
    points = line.strip().split(" ")
    x = points[0]
    y = points[1]
    dataset2[ind][0] = x
    dataset2[ind][1] = y
    ind += 1

# print("x: ", dataset[:,0])
# print("y: ", dataset[:,1])
# plt.scatter(dataset[:,0], dataset[:,1])
# plt.show()

Running GMM on 3gaussian dataset

In [37]:
gmm = GaussianMixtureModel(k=3)
mean, cov, pi = gmm.fit(dataset2)


print("Mean 1: ", mean[0], " Covariance 1: ", cov[0], " N1", pi[0])
print("\nMean 2: ", mean[1], " Covariance 2: ", cov[1], " N2", pi[1])
print("\nMean 3: ", mean[2], " Covariance 3: ", cov[2], " N3", pi[2])



Mean 1:  [7.021539   4.01544854]  Covariance 1:  [[0.99045127 0.50097416]
 [0.50097416 0.99565041]]  N1 0.29843951629533266

Mean 2:  [5.01167003 7.00142028]  Covariance 2:  [[0.97978461 0.18520241]
 [0.18520241 0.97461388]]  N2 0.4959898380406252

Mean 3:  [3.0395831  3.04817572]  Covariance 3:  [[1.02841795 0.02661995]
 [0.02661995 3.38406276]]  N3 0.20557064566404173
